In [ ]:
# UPDATED: Uses model_tbyt_3_withnormalization.py
import torch
import os
from model_tbyt_3_withnormalization import GPT, GPTConfig
from checkpoint_utils import load_checkpoint

In [ ]:
# UPDATED: Loading logic moved here
checkpoint_path = os.path.join(os.getcwd(), 'Grid_training_without_duplicates/Final_N256_K16_L2_H1_E32_r8over1_npos1_mlp1_dup0_testK16_iters60000.pt')
device = 'cpu'
model, config = load_checkpoint(checkpoint_path, device=device)

vocab_n = config.vocab_size - 1
block_size = config.block_size

batch_size = 1
def get_batch(changing_num=-1, changing_index=-1, initial_sequence=None, batch_size=batch_size):
   def cat_sorted_tensor(x):
      if initial_sequence is not None:
         x = initial_sequence
      else:
         x = x
      if changing_num != -1:
         if changing_index == -1:
            x[0] = changing_num
         else:
            x[changing_index] = changing_num
      vals, _ = torch.sort(x)
      # Use vocab_n as separator
      return torch.cat((x, torch.tensor([vocab_n]), vals), dim=0)
   x = torch.stack([cat_sorted_tensor(torch.randperm(vocab_n)[:block_size]) for _ in range(batch_size)])
   return x

In [ ]:
# UPDATED: Inference with new model
idx = get_batch()
print('idx dim is ', idx.shape)
logits, loss = model(idx)
if loss is not None:
    print('loss is ', loss.item())
print(f'idx is: {idx}')
print('model output is ', torch.argmax(logits, dim=-1))